In [19]:
!pip install keras-core keras-nlp tensorflow scikit-learn matplotlib

import keras_core as keras
import keras_nlp
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd



In [22]:
data = pd.read_csv('/content/SC_Vuln_8label.csv')
X_data = data['code']  # Change this to match the actual column name in your dataset
y_data = data['label']

encoder = LabelEncoder()
y_data_encoded = encoder.fit_transform(y_data)

assert len(encoder.classes_) == 8, "The dataset should contain exactly 8 classes."
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data_encoded, test_size=0.2, random_state=42)

print("\nTraining data samples:")
print(X_train.head())
print("\nTraining labels:")
print(y_train[:5])



Training data samples:
817     pragma solidity ^0.4.15;\n/**\n * @title ERC20...
3585    // Copyright (C) 2020  Argent Labs Ltd. <https...
2525    pragma solidity ^0.4.24;\n\n\n \n \n \n \ncont...
2540    pragma solidity ^0.4.24;\n\ncontract Random\n{...
2570    pragma solidity ^0.4.9;\n\ncontract SafeMath {...
Name: code, dtype: object

Training labels:
[5 7 0 0 0]


In [23]:
print("\nLabel Distribution in the Full Dataset:")
print(data['label'].value_counts())



Label Distribution in the Full Dataset:
label
./Dataset/reentrancy (RE)/                1218
./Dataset/unchecked external call (UC)    1199
./Dataset/integer overflow (OF)/           590
./Dataset/block number dependency (BN)     406
./Dataset/ether strict equality (SE)       366
./Dataset/timestamp dependency (TP)/       312
./Dataset/dangerous delegatecall (DE)/      97
./Dataset/ether frozen (EF)                 97
Name: count, dtype: int64


In [24]:
preset = "deberta_v3_base_en"
sequence_length = 256

# Set up preprocessor
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(preset, sequence_length=sequence_length)
# Initialize classifier with pre-trained weights
classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
    preset,
    preprocessor=preprocessor,
    num_classes=4  # Set the number of output classes
)


In [25]:
classifier.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-5),
    metrics=["accuracy"]
)
